In [ ]:
import torch
from transformers import AutoImageProcessor, AutoModel
from transformers.image_utils import load_image
import os
from PIL import Image


from huggingface_hub import login
from keys import token
login(token=token)

image_path = r"D:\KIMKEK-EXTRA\dataset_validation\scenes\03_Cam2_MorningPeak\images\frame360.png"
assert os.path.isfile(image_path), f"Image not found: {image_path}"
image = load_image(image_path)
print("Image size:", image.height, image.width)

# # Use a pipeline as a high-level helper
# from transformers import pipeline
# pipe = pipeline("image-feature-extraction", model="facebook/dinov3-vitl16-pretrain-lvd1689m")

from transformers import ViTImageProcessor, AutoModel

processor = ViTImageProcessor.from_pretrained("facebook/dinov3-vitl16-pretrain-lvd1689m")
model = AutoModel.from_pretrained("facebook/dinov3-vitl16-pretrain-lvd1689m")


patch_size = model.config.patch_size
print("Patch size:", patch_size) # 16
print("Num register tokens:", model.config.num_register_tokens) # 4

inputs = processor(images=image, return_tensors="pt")
print("Preprocessed image size:", inputs.pixel_values.shape)  # [1, 3, 224, 224]

batch_size, _, img_height, img_width = inputs.pixel_values.shape
num_patches_height, num_patches_width = img_height // patch_size, img_width // patch_size
num_patches_flat = num_patches_height * num_patches_width

print(inputs)

with torch.inference_mode():
  outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
print(last_hidden_states.shape)  # [1, 1 + 4 + 256, 384]
assert last_hidden_states.shape == (batch_size, 1 + model.config.num_register_tokens + num_patches_flat, model.config.hidden_size)

cls_token = last_hidden_states[:, 0, :]
patch_features_flat = last_hidden_states[:, 1 + model.config.num_register_tokens:, :]
patch_features = patch_features_flat.unflatten(1, (num_patches_height, num_patches_width))

print("cls shape:", cls_token.shape)
print("patch features shape:", patch_features.shape)

Image size: 1677 1514
Patch size: 16
Num register tokens: 4
Preprocessed image size: torch.Size([1, 3, 224, 224])
{'pixel_values': tensor([[[[-0.4739, -0.0458, -0.0458,  ...,  0.9132,  0.7077,  0.9817],
          [-1.0904, -1.2445, -1.2788,  ...,  0.8789,  0.7933,  0.8447],
          [-1.5014, -1.4158, -1.4843,  ...,  0.8618,  0.6563,  0.5878],
          ...,
          [-0.8678, -0.8678, -0.9020,  ..., -1.4672, -1.4500, -1.4329],
          [-0.8849, -0.8678, -0.8507,  ..., -1.4500, -1.4500, -1.4500],
          [-0.8507, -0.8335, -0.8335,  ..., -1.4500, -1.4500, -1.4672]],

         [[ 0.1176,  0.5553,  0.5728,  ...,  0.8704,  0.7479,  1.0805],
          [-0.4601, -0.6176, -0.7227,  ...,  0.8529,  0.8004,  0.8704],
          [-1.0553, -1.0028, -1.0378,  ...,  0.7654,  0.6604,  0.6078],
          ...,
          [-0.4601, -0.4601, -0.4951,  ..., -1.1604, -1.1429, -1.1429],
          [-0.4776, -0.4601, -0.4426,  ..., -1.1429, -1.1429, -1.1604],
          [-0.4426, -0.4251, -0.4251,  ..., -

In [2]:
# import torch

# REPO_DIR = r'..\dinov3'

# # DINOv3 ViT models pretrained on web images
# model = torch.hub.load(REPO_DIR, 'dinov3_vitl16', source='local', weights=r'..\dinov3_vitl16_pretrain_lvd1689m-8aa4cbdd.pth')

# patch_size = model.config.patch_size
# print("Patch size:", patch_size) # 16
# print("Num register tokens:", model.config.num_register_tokens) # 4

# inputs = processor(images=image, return_tensors="pt")
# print("Preprocessed image size:", inputs.pixel_values.shape)  # [1, 3, 224, 224]

# batch_size, _, img_height, img_width = inputs.pixel_values.shape
# num_patches_height, num_patches_width = img_height // patch_size, img_width // patch_size
# num_patches_flat = num_patches_height * num_patches_width

# with torch.inference_mode():
#   outputs = model(**inputs)

# last_hidden_states = outputs.last_hidden_state
# print(last_hidden_states.shape)  # [1, 1 + 4 + 256, 384]
# assert last_hidden_states.shape == (batch_size, 1 + model.config.num_register_tokens + num_patches_flat, model.config.hidden_size)

# cls_token = last_hidden_states[:, 0, :]
# patch_features_flat = last_hidden_states[:, 1 + model.config.num_register_tokens:, :]
# patch_features = patch_features_flat.unflatten(1, (num_patches_height, num_patches_width))

In [3]:
print(dino)
out = dino(image)

NameError: name 'dino' is not defined